Imports

In [1]:
%load_ext autoreload
%autoreload 2
from pathlib import Path
import numpy as np
import matplotlib as plt
import torch

In [2]:
torch.cuda.is_available()

True

Download datasets

In [9]:
# add download of ModelNet40 in data/ModelNet40
print('Downloading ModelNet40...')
!wget modelnet.cs.princeton.edu/ModelNet40.zip -P data
print('Extracting ModelNet40...')
!unzip -q data/ModelNet40.zip -d data
!rm data/ModelNet40.zip
print('ModelNet40 done')

--2021-12-12 22:01:08--  http://modelnet.cs.princeton.edu/ModelNet40.zip
Resolving modelnet.cs.princeton.edu (modelnet.cs.princeton.edu)... 128.112.136.61
Connecting to modelnet.cs.princeton.edu (modelnet.cs.princeton.edu)|128.112.136.61|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2039180837 (1.9G) [application/zip]
Saving to: ‘data/ModelNet40.zip’

ModelNet40.zip      100%[===================>]   1.90G  3.24MB/s    in 45m 0s  

2021-12-12 22:46:08 (738 KB/s) - ‘data/ModelNet40.zip’ saved [2039180837/2039180837]

Extracting ModelNet40...
ModelNet40 done
--2021-12-12 22:47:04--  http://download.microsoft.com/download/2/8/5/28564B23-0828-408F-8631-23B1EFF1DAC8/tsdf.zip
Resolving download.microsoft.com (download.microsoft.com)... 2a02:26f0:3500:595::e59, 2a02:26f0:3500:583::e59, 23.213.164.137
Connecting to download.microsoft.com (download.microsoft.com)|2a02:26f0:3500:595::e59|:80... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Loc

Download 7Scenes from https://drive.google.com/file/d/1XdQ3muo5anFA28ZFch06z_iTEjXxLEQi/view and extract into data/7Scene

In [5]:
from data.SevenScenes import SevenScenes

train_dataset = SevenScenes('train')
test_dataset = SevenScenes('test')

# Get length, which is a call to __len__ function
print(f'Length of train set: {len(train_dataset)}')  # expected output: 296
# Get length, which is a call to __len__ function
print(f'Length of val set: {len(test_dataset)}')  # expected output: 57

Length of train set: 296
Length of val set: 57


In [6]:
train_sample = train_dataset[3]
print(f'Pointcloud A: {train_sample["pointcloud_a"].shape}')
print(f'Pointcloud B: {train_sample["pointcloud_b"].shape}')
print(f'Transformation Matrix: {train_sample["transformation_matrix"]}')
print(f'Class: {train_sample["class"]}')

Pointcloud A: torch.Size([3, 2048])
Pointcloud B: torch.Size([3, 2048])
Transformation Matrix: tensor([[ 0.8269,  0.0630,  0.5588,  0.0387],
        [-0.0185,  0.9962, -0.0848, -0.0781],
        [-0.5620,  0.0598,  0.8250, -0.3328],
        [ 0.0000,  0.0000,  0.0000,  1.0000]])
Class: 7-scenes-chess


## Run simple overfit configuration

In [7]:
from data.ModelNet40 import ModelNet40DataModule
modelnet = ModelNet40DataModule(batch_size=16)
modelnet.setup()

In [10]:
from data.SevenScenes import SevenScenesDataModule
sevenscenes = SevenScenesDataModule(batch_size=16)
sevenscenes.setup()

In [11]:
import pytorch_lightning as pl
from training.trainer import GenReg, GenRegVal

config = {
    'lr_D': 0.01,
    'lr_G': 0.01}

trainer = pl.Trainer(max_epochs=1, gpus=1, overfit_batches=0.1)
# ModelNet40
# trainer.fit(model=GenRegVal(params=config), train_dataloaders=modelnet.train_dataloader(), val_dataloaders=modelnet.val_dataloader())
# 7 Scenes
trainer.fit(model=GenReg(params=config), train_dataloaders=sevenscenes.train_dataloader())

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name    | Type         | Params
-----------------------------------------
0 | G_model | DataParallel | 5.9 M 
1 | D_model | DataParallel | 1.7 M 
-----------------------------------------
7.6 M     Trainable params
0         Non-trainable params
7.6 M     Total params
30.254    Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 2            |        cudaMalloc retries: 2         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |    3773 MB |    4399 MB |    6901 MB |    3128 MB |
|       from large pool |    3759 MB |    4389 MB |    6885 MB |    3126 MB |
|       from small pool |      14 MB |      14 MB |      16 MB |       1 MB |
|---------------------------------------------------------------------------|
| Active memory         |    3773 MB |    4399 MB |    6901 MB |    3128 MB |
|       from large pool |    3759 MB |    4389 MB |    6885 MB |

RuntimeError: CUDA out of memory. Tried to allocate 128.00 MiB (GPU 0; 5.80 GiB total capacity; 4.33 GiB already allocated; 9.44 MiB free; 4.49 GiB reserved in total by PyTorch)

Visualize results

In [8]:
# add visualization